In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q accelerate==0.20.3 torch==2.2.1
! pip install -q -U transformers peft shap
! pip install torch datasets
! pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os,torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer
)
from peft import AutoPeftModelForSequenceClassification
from torch.utils.data import Dataset
from datasets import Dataset, DatasetDict


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Importing the final model:

In [ ]:
save_path = '/content/drive/My Drive/CBS/CBS Thesis Lydia & Sara/Models/mistral_0405'

model = AutoPeftModelForSequenceClassification.from_pretrained(save_path,
                                                               num_labels=8,
                                                               problem_type="multi_label_classification",
                                                               use_auth_token='') #insert HF token

tokenizer = AutoTokenizer.from_pretrained(save_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id

merged_model = model.merge_and_unload()

Importing the data:

In [ ]:
path='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/03_Data_Modeling/full_preprocessed_XX.csv' #adjust to current language
df_temp= pd.read_csv(path, sep='\t', encoding='utf-16')

#Get 20 000 random rows for predictions
df = df_temp.sample(n=20000, random_state=42)

#Get 200 random rows for SHAP analysis
df_shap = df_temp.sample(200, random_state=42)


Create HF dataset:

In [ ]:
MAX_LEN=512

#Same set-up as in the Classification notebook
class CustomHFDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, text_column, device=None):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.text_column = text_column
        self.max_len = max_len
        self.device = device if device is not None else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        record = self.dataset[index]
        text = str(record[self.text_column])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )

        return {
            "input_ids": inputs["input_ids"].flatten(),
            "attention_mask": inputs["attention_mask"].flatten()
        }


dataset = Dataset.from_pandas(df)
custom_dataset = CustomHFDataset(dataset, tokenizer,
                                 max_len=MAX_LEN, text_column='Translation')

Perform Predictions on the 20k sample:

In [ ]:
model=model.to(device)
model.eval()

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(custom_dataset, batch_size=16, shuffle=False)

all_predictions = []
all_probabilities = []

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Processing batches"):
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels' and k != 'token_type_ids'}
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.sigmoid(logits).cpu().numpy()
        predicted_labels = (probabilities > 0.5).astype(float)

        # Store predictions and probabilities
        all_probabilities.extend(probabilities)
        all_predictions.extend(predicted_labels)

all_probabilities = np.array(all_probabilities)
all_predictions = np.array(all_predictions)
list_of_predictions = all_predictions.tolist()

In [ ]:
df['predicted_label'] = list_of_predictions

#Save the dataframe with the predictions-column
save_path='/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/20k....csv' #change accordingly
df.to_csv(save_path, sep='\t', encoding='utf-16', index=False)

# SHAP Analysis

In [ ]:
import shap
from tqdm.auto import tqdm
from transformers import pipeline
import pickle
from collections import defaultdict
import numpy as np

In [ ]:
merged_model=merged_model.to(device)
merged_model.eval()

SHAP calculations:

In [ ]:
inference_pipeline = pipeline(
    tokenizer=tokenizer,
    task="text-classification",
    model=merged_model,
    max_length=512,
    return_all_scores=True,
    padding=True,
    truncation=True,
    device=device
)


batch_size = 64
text_batches = [df_shap["Translation"][i:i + batch_size] for i in range(0, df_shap["Translation"].shape[0], batch_size)]

explainer = shap.Explainer(inference_pipeline,
                           inference_pipeline.tokenizer)

shap_values_list = []

for text_batch in tqdm(text_batches, desc="Explaining"):
    shap_values = explainer(text_batch.tolist())
    shap_values_list.extend(shap_values)


In [ ]:
#Save the SHAP values

file_path = '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/SHAP_200_XX.pkl' #adjust accordingly

with open(file_path, 'wb') as file:
    pickle.dump(shap_values_list, file)

print("SHAP values saved successfully.")

In [ ]:
#Plotting a shap plot from the shap_values_list
shap.plots.text(shap_values_list[1])

SHAP analysis of all 1000 sampled tweets:

In [ ]:
file_path_SE= '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/SHAP_200_SE.pkl'
file_path_DA= '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/DA_shap_values.pkl'
file_path_FI= '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/FI_shap_values.pkl'
file_path_DE= '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/DE_shap_values.pkl'
file_path_EN= '/content/drive/MyDrive/CBS/CBS Thesis Lydia & Sara/Data/04_Data_Analysis/EN_shap_values.pkl'

with open(file_path_DA, 'rb') as file:
    DA_shap_values_loaded = pickle.load(file)

with open(file_path_FI, 'rb') as file:
    FI_shap_values_loaded = pickle.load(file)

with open(file_path_SE, 'rb') as file:
    SE_shap_values_loaded = pickle.load(file)

with open(file_path_DE, 'rb') as file:
    DE_shap_values_loaded = pickle.load(file)

with open(file_path_EN, 'rb') as file:
    EN_shap_values_loaded = pickle.load(file)

all_shap_values = DA_shap_values_loaded + FI_shap_values_loaded + SE_shap_values_loaded + DE_shap_values_loaded + EN_shap_values_loaded

In [ ]:
label_names = [
    'Not Categorisable',
    'Conflict & Crisis',
    'Migration Flow',
    'Host Country Security',
    'Host Country Politics',
    'Refugee Rights & Advocacy',
    'Host Country Resources',
    'Host Country Symbolic Discourse'
]

# Dictionaries for the SHAP value sums and counts for each label
token_shap_sums = defaultdict(lambda: defaultdict(float))
token_counts = defaultdict(lambda: defaultdict(int))

# Collecting the SHAP values
for shap_values in all_shap_values:
    tokens = shap_values.data.flatten()
    for label_idx, label_name in enumerate(label_names):
        class_shap_values = shap_values.values[..., label_idx].flatten()
        for token, shap_value in zip(tokens, class_shap_values):
            token_shap_sums[token][label_name] += shap_value
            token_counts[token][label_name] += 1

total_token_counts = {token: sum(counts.values()) for token, counts in token_counts.items()}

# Average SHAP value for each token per label (if the token has 3 or more occurences)
token_info = {
    token: {
        label: token_shap_sums[token][label] / token_counts[token][label]
        for label in label_names if token_counts[token][label] > 0
    }
    for token in token_counts if total_token_counts[token] >= 3
}

In [ ]:
class_index = 7 #Choose class of interest

target_label = label_names[class_index]

# Sorting tokens by average SHAP value for the class_index label
sorted_tokens = sorted(token_info.items(), key=lambda x: x[1].get(target_label, 0), reverse=True)

#Take the top 10 of these
N = 10
top_tokens = sorted_tokens[:min(N, len(sorted_tokens))]


data = {
    'Token': [token for token, _ in top_tokens],
    **{
        label: [f"{token_info[token].get(label, 'n/a'):.3f}" if label in token_info[token] else 'n/a'
                for token, _ in top_tokens]
        for label in label_names
    }
}

df = pd.DataFrame(data)
df.set_index('Token', inplace=True)

In [ ]:
print(f"Top {len(top_tokens)} tokens by average SHAP value impact for '{target_label}':")
df.head(10)